## detect Anomaly in data

In [3]:
import json
import pandas as pd

In [4]:
with open('stores.json') as data:
    stores_json = json.load(data)

In [5]:
stores = pd.DataFrame(stores_json)
stores.head()

,Banner,Region,Store ID
0,Walmart,Northern California,66999
1,Trader Joes,Northern California,4698
2,Safeway,Northern California,39482
3,Whole Foods,Northern California,34957
4,Walmart,New York,12837


In [7]:
prices=pd.read_csv('prices.csv')
prices.head()

,Auditor ID,Date,Price,Store ID,UPC
0,234,10/18/2017,24.95,66999,268588472
1,234,10/27/2017,49.71,66999,475245085
2,234,10/20/2017,25.75,66999,126967843
3,234,10/23/2017,18.81,66999,708930835
4,234,10/23/2017,33.32,66999,325885139


In [8]:
#combining price and store 
combined = prices.merge(stores, 'left', 'Store ID')

In [9]:
print(len(prices))
print(len(stores))
print(len(combined))

12315
28
12315


In [10]:
combined.head(10)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region
0,234,10/18/2017,24.95,66999,268588472,Walmart,Northern California
1,234,10/27/2017,49.71,66999,475245085,Walmart,Northern California
2,234,10/20/2017,25.75,66999,126967843,Walmart,Northern California
3,234,10/23/2017,18.81,66999,708930835,Walmart,Northern California
4,234,10/23/2017,33.32,66999,325885139,Walmart,Northern California
5,234,10/16/2017,40.43,66999,509419735,Walmart,Northern California
6,234,10/22/2017,35.10,66999,736362595,Walmart,Northern California
7,234,10/21/2017,33.65,66999,652377244,Walmart,Northern California
8,234,10/16/2017,22.30,66999,277394644,Walmart,Northern California
9,234,10/20/2017,12.32,66999,996849471,Walmart,Northern California


In [11]:
combined.isnull().sum()

Auditor ID      0
Date            0
Price           0
Store ID        0
UPC             0
Banner        804
Region        804
dtype: int64

In [12]:
missing_stores = combined[combined['Banner'].isnull()]

- There are some stores that sell products; however, these stores do not exist in "stores.csv", which is actually a store dictionary. This dictionary is supposed to include all store IDs.

In [14]:
missing_stores.head(10)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region
813,536,10/19/2017,48.69,60957,340260209,NaN,NaN
814,536,10/29/2017,55.79,60957,749133422,NaN,NaN
815,536,10/25/2017,5.99,60957,16999755,NaN,NaN
816,536,10/28/2017,11.09,60957,673299284,NaN,NaN
817,536,10/27/2017,32.79,60957,204071291,NaN,NaN
818,536,10/20/2017,11.79,60957,736014357,NaN,NaN
819,536,10/27/2017,15.19,60957,252888653,NaN,NaN
820,536,10/21/2017,43.79,60957,676939567,NaN,NaN
821,536,10/23/2017,44.59,60957,433536637,NaN,NaN
822,536,10/18/2017,60.29,60957,83141336,NaN,NaN


In [15]:
missing_stores['Store ID'].nunique()

2

In [16]:
missing_stores['Store ID'].drop_duplicates()

813     60957
4109    38472
Name: Store ID, dtype: int64

- Store 38472 and Store 60957 do not exist in store.csv.  

In [18]:
missing_stores.groupby(by='Store ID')['UPC'].count()

Store ID
38472    503
60957    301
Name: UPC, dtype: int64

- 503 products were sold in Store 38472 and 301 products were sold in Store 60957.
- In the final output, these products will not exist since 'Banner' column is required in the final output.

In [19]:
combined_final = combined.dropna()

In [20]:
combined_final.isnull().sum()

Auditor ID    0
Date          0
Price         0
Store ID      0
UPC           0
Banner        0
Region        0
dtype: int64

In [21]:
print(len(combined))
print(len(combined_final))

12315
11511


- 804 products (12315-1151 = 804) were dropped, which is consisted what we found above (503 + 301 = 804)

In [22]:
### Creating pivot table
pivot = combined_final.pivot_table(values = 'Price', index = ['Banner', 'UPC'], columns = 'Region', aggfunc='mean')

In [1]:
206708/101843

2.0296731243188044

In [2]:
73620/58386

1.2609187133901962

- Since the indeces of final output are Banner and UPC, we need to do aggregation, which is 'mean' in that case.
- Actually, prices of stores of a banner is averaged. 
- Based on the subject matter expertise or desired analysis, median might be another option for aggregation.
- When checked stores.csv, each region has one store ID for each banner. So, mean or median, or any aggregation function, will not make any difference in this question.

In [23]:
pivot.head()

Region            Kansas  New York  Northern California  Texas
Banner  UPC                                                   
Safeway 11873171     NaN       NaN                 6.09   5.19
        15052612   53.99       NaN                  NaN  54.49
        16482322   17.89       NaN                  NaN  18.09
        16729338    7.99       NaN                 9.39   8.09
        16829288    3.59       NaN                 4.19   3.59

In [24]:
pivot.fillna('-', inplace = True)
pivot.head()

Region           Kansas New York Northern California  Texas
Banner  UPC                                                
Safeway 11873171      -        -                6.09   5.19
        15052612  53.99        -                   -  54.49
        16482322  17.89        -                   -  18.09
        16729338   7.99        -                9.39   8.09
        16829288   3.59        -                4.19   3.59

In [25]:
## Reordering columns as requested in the question
pivot = pivot[['Northern California', 'New York', 'Kansas', 'Texas']]
pivot.head()

Region           Northern California New York Kansas  Texas
Banner  UPC                                                
Safeway 11873171                6.09        -      -   5.19
        15052612                   -        -  53.99  54.49
        16482322                   -        -  17.89  18.09
        16729338                9.39        -   7.99   8.09
        16829288                4.19        -   3.59   3.59

In [26]:
## Final output
pivot.to_csv('final_output.csv', index = True)

### Extra Exploration

In [27]:
## Count of products sold in stores
combined.groupby(by=['Region','Banner','Store ID'])['UPC'].count()

## These excludes sales in Stores 38472 and 60957 since we can retrive banners for them.

Region               Banner       Store ID
Kansas               Safeway      39485       819
                     Trader Joes  29384       381
                     Walmart      40593       585
                     Wegmans      3948        389
                     Whole Foods  39287       913
New York             Trader Joes  9487        382
                     Walmart      12837       859
                     Wegmans      2938        933
                     Whole Foods  50948       851
Northern California  Safeway      39482       306
                     Trader Joes  4698        401
                     Walmart      66999       412
                     Whole Foods  34957       515
Texas                Safeway      29382       838
                     Trader Joes  40586       898
                     Walmart      50495       579
                     Wegmans      98638       927
                     Whole Foods  98736       523
Name: UPC, dtype: int64

In [28]:
auditor = pd.read_csv('auditors.csv')

In [29]:
auditor

,Auditor ID,First,Last,Region
0,234,Sue,Smith,Northern California
1,536,Bob,Smith,Northern California
2,98,Jack,Smith,New York
3,203,Jill,Smith,New York
4,304,Jerry,Johnson,Texas
5,63,Randy,Johnson,Texas
6,1326,Mike,Johnson,Kansas
7,713,Dave,Johnson,Kansas


- Though Hawaii exists in stores.csv, there is no record related to Hawaii in prices.csv. That mean no auditor is assigned to Hawaii.

In [36]:
#Following analysis excludes missing stores

In [31]:
### Count of UPC each auditor collected
combined_final.groupby(by = ['Auditor ID'])['UPC'].count()

Auditor ID
63      1477
98      2090
203     1241
234      412
304     2288
536      916
713     1302
1326    1785
Name: UPC, dtype: int64

In [32]:
## Count of UPC collected by each auditor in region
combined_final.groupby(by = ['Region','Auditor ID'])['UPC'].count()

Region               Auditor ID
Kansas               713           1302
                     1326          1785
New York             98            1784
                     203           1241
Northern California  98             306
                     234            412
                     536            916
Texas                63            1477
                     304           2288
Name: UPC, dtype: int64

######  All auditors were assigned to collect prices. Only auditor 98 was sent to two different regions.

In [34]:
### Count of UPCs collected in each region
combined_final.groupby(by = ['Region'])['UPC'].count()

Region
Kansas                 3087
New York               3025
Northern California    1634
Texas                  3765
Name: UPC, dtype: int64

In [35]:
combined_final['UPC'].nunique()

1000

## Anomalies

- Though Hawaii exists in stores.csv, there is no record related to Hawaii in prices.csv. That mean no auditor is assigned to Hawaii.
- There are missnig region multiplers, which means it would not be possible to calculate price. The way to handling missing data is to get median or mean - domain experts can act upon.
- Similarly, since there are missing store IDs in stores.csv, we dropped UPCs which were missing store IDs in stores.csv to create the final output csv. Hence, it is not possible to calculate base price for those UPCs - assuming that we have store multipliers and region multiplers.